In [1]:
import torch


device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
    
device

/Users/cgiudice/facu/aprendizaje_automatico/aaut-catdog/.venv/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'cpu'

In [2]:
from catdog.dataset import CatDogDataset

In [3]:
DATA_DIR = '../data/'

In [4]:
import pandas as pd


train_set_pd = pd.read_csv(DATA_DIR + 'train.csv')
train_set_torch = CatDogDataset(train_set_pd)
print(f"Loaded train set with length {len(train_set_torch)}")

test_set_pd = pd.read_csv(DATA_DIR + 'test.csv')
test_set_torch = CatDogDataset(test_set_pd)
print(f"Loaded test set with length {len(test_set_torch)}")

Loaded train set with length 2948
Loaded test set with length 738


In [5]:
type(test_set_torch[0][1])

numpy.int64

In [6]:
batch_size = 32
train_loader = torch.utils.data.DataLoader(train_set_torch, batch_size=batch_size) # shuffle=True, num_workers=2
test_loader = torch.utils.data.DataLoader(test_set_torch, batch_size=batch_size) # shuffle=True, num_workers=2


In [7]:
import numpy as np


seed = 142
torch.manual_seed(seed)
np.random.seed(seed)

#### test image

import numpy as np
from PIL import Image


test_img_path = DATA_DIR + 'images/' + cat_dog_df["file"].iloc[0]
test_img = Image.open(test_img_path).convert("RGB")

In [8]:
from catdog.models.feedforward import MLPClassifier


model = MLPClassifier(input_size=500*500*3)
model.to(device)

/Users/cgiudice/facu/aprendizaje_automatico/aaut-catdog/.venv/lib/python3.7/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


MLPClassifier(
  (AUROC): AUROC()
  (model): Sequential(
    (0): CatDogOutput(
      (classify_layer): Linear(in_features=750000, out_features=1, bias=True)
      (bbox_layer): Linear(in_features=750000, out_features=4, bias=True)
    )
  )
)

In [9]:
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint


seed_everything(142, workers=True)

checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    dirpath='./lightning_checkpoints/',
    filename='test_model_training',
    mode='min',
    save_top_k=3
)

trainer = Trainer(gpus=int(device=='cuda'), # hacky way to say 0 or 1 
                  max_epochs=40,
                  callbacks=[EarlyStopping(monitor="val_loss", patience=15, mode='min'),
                             checkpoint_callback])
trainer.fit(model, train_loader, test_loader)  # here we are using test set as validation set

Global seed set to 142
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name  | Type       | Params
-------------------------------------
0 | AUROC | AUROC      | 0     
1 | model | Sequential | 3.8 M 
-------------------------------------
3.8 M     Trainable params
0         Non-trainable params
3.8 M     Total params
15.000    Total estimated model params size (MB)


Validation sanity check:   0%|                                                                                                                         | 0/2 [00:00<?, ?it/s]

/Users/cgiudice/facu/aprendizaje_automatico/aaut-catdog/.venv/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


RuntimeError: mat1 and mat2 shapes cannot be multiplied (48000x500 and 750000x1)